In [ ]:
!wget --no-check-certificate \
  https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip 
  

--2021-07-31 23:34:04--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.133.22
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.133.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1061576029 (1012M) [binary/octet-stream]
Saving to: ‘dataset52a7b21.zip’

dataset52a7b21.zip  100%[===================>]   1012M  17.2MB/s    in 60s     

2021-07-31 23:35:05 (16.8 MB/s) - ‘dataset52a7b21.zip’ saved [1061576029/1061576029]



In [ ]:
import numpy as np
import csv
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam#Try all of these
from sklearn.preprocessing import LabelEncoder

In [ ]:
print(tf.__version__)


2.5.0


In [ ]:
!unzip dataset52a7b21.zip

Archive:  dataset52a7b21.zip
   creating: dataset/
  inflating: dataset/train.csv       
  inflating: dataset/sample_submission.csv  
  inflating: dataset/test.csv        
  inflating: dataset/.~lock.train.csv#  


In [ ]:
train_df=pd.read_csv('dataset/train.csv',escapechar = "\\",quoting = csv.QUOTE_NONE)

In [ ]:
train_df.fillna(train_df.mode().iloc[0],inplace=True)

In [ ]:
df1['text']=df1['TITLE']+" "+df1['DESCRIPTION']+" "+df1['BRAND']
df1=df1.drop(columns=['TITLE','DESCRIPTION','BRAND'])
df1

,BULLET_POINTS,BROWSE_NODE_ID,text
0,"[Pete the Cat Bedtime Blues plush doll,Based o...",0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch Pet..."
1773215,[Chelsea doll and her friends can play out so ...,0,"Barbie Chelsea Assortment 2, Multi Color Barbie"
1694581,"[Condition- New,Package includes- 1 pcs only,F...",0,ELECTROPRIME Realistic Vinyl Baby Girl Doll Li...
2294866,[From Manhattan Toy's multiple award-winning G...,0,Manhattan Toy Groovy Girls Style Scents Lilly ...
1469000,,0,ZWSISU 7 Outfits American Girl Doll Clothes Ac...
...,...,...,...
2801983,"[Requires Only 1 Coat,Write on In 2 Days,No Mi...",2801983,Writeyboard 50 Square Feet 1-Part White Dry Er...
2830573,,2830573,"Journal of Inventions, Leonardo Da Vinci"
2847632,"[Material: 88% Cotton, 12% Lycra | Type: Full ...",2847632,NO FALL Men's Anti-slip Cotton Full Length Soc...
2860317,[This color-coded bench scraper is an excellen...,2860317,"Remco 69625 Stiff Hand Scraper, Injection Mold..."


In [ ]:
import re
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import string

from nltk import stem
from nltk.corpus import treebank
stems = nltk.PorterStemmer()

stop_words = stopwords.words('english')
more=['cm','x','inches','inch','length','width','size','shape','height']
stop_words=stop_words+more

def cleaning(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
def swrem(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text

def stemming(text):
    text = ' '. join(stems.stem(word) for word in text.split())
    return text
#another cleaning method for BULLET_POINTS
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#   text = re.sub(r'\W+', '', text)
    return text


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df1['text']=df1['text'].apply(cleaning)
df1['text']=df1['text'].apply(swrem)
df1['text']=df1['text'].apply(stemming)
df1['BULLET_POINTS']=df1['BULLET_POINTS'].apply(swrem)
df1['BULLET_POINTS']=df1['BULLET_POINTS'].apply(clean_text)
df1['BULLET_POINTS']=df1['BULLET_POINTS'].apply(stemming)
df1

KeyboardInterrupt: ignored

In [ ]:
df1['text']=df1['text']+" "+df1['BULLET_POINTS']
df1= df1.drop(colums=['BULLET_POINTS'])